In [1]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [2]:
train_df = pd.read_csv('/home/karen/Documents/Datasets/Disaster_tweets/train.csv')
test_df = pd.read_csv('/home/karen/Documents/Datasets/Disaster_tweets/test.csv')
submission_df = pd.read_csv('/home/karen/Documents/Datasets/Disaster_tweets/sample_submission.csv')

In [3]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [6]:
#Example of disaster tweet
train_df[train_df['target'] == 1]['text'].values[1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
#Non disaster tweet
train_df[train_df['target'] == 0]['text'].values[1]

'I love fruits'

### Vectorization using count_vectorizer

In [8]:
#Build vectors 
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

#get counts for the first 5 tweets in the data
example_vectors = count_vectorizer.fit_transform(train_df['text'][:5])

In [10]:
# we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_vectors[0].todense().shape)
print(example_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


In [11]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

# note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

In [12]:
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier()

In [13]:
# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
# scores

array([0.59453669, 0.5642787 , 0.64082434])

In [21]:
# clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

### Vectorization usinf TF-IDF

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [26]:
tfidf_train_vectors = tfidf_vectorizer.fit_transform(train_df["text"])

tfidf_test_vectors = tfidf_vectorizer.transform(test_df["text"])

In [28]:
clf.fit(tfidf_train_vectors, train_df["target"])

RidgeClassifier()

In [29]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, tfidf_train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.63366337, 0.6122449 , 0.68407835])

In [31]:
submission_df["target"] = clf.predict(tfidf_test_vectors)

In [32]:
submission_df

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [33]:
submission_df.to_csv("submission.csv", index=False)